# Hands-On Lab: Gaussian Process Regression for Molecular Energy Prediction

**Learning Objectives:**
- Understand Gaussian Process Regression (GPR) fundamentals
- Implement different kernel functions from scratch
- Compare kernel performance on real molecular data
- Learn hyperparameter optimization through Bayesian inference

**Pedagogical Flow:**
1.  **Explore** - Understand the data and problem
2.  **Build** - Implement simple kernel functions
3.  **Enhance** - Add complexity with full GPR class
4.  **Compare** - Test different kernels
5.  **Reflect** - Analyze and discuss results

---

## Setup: Installing Required Libraries

In [ ]:
#!pip install scipy numpy matplotlib -q

In [ ]:
from scipy.optimize import minimize
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

---
# Part 1:  EXPLORE - Understanding the Data

We're working with molecular energy data where:
- **X (R)**: Atomic distances in Angstroms
- **Y (E_FCI)**: Full Configuration Interaction energies in Hartree units

Our goal: **Predict energy at any distance using only 16 training points**

In [ ]:
# Download H2 molecule data
!wget -q https://raw.githubusercontent.com/dralgroup/MLinQCbook22-NN/main/R_451.dat
!wget -q https://raw.githubusercontent.com/dralgroup/MLinQCbook22-NN/main/E_FCI_451.dat
print("Dataset downloaded!")
print("- R_451.dat: Internuclear distances (451 points)")
print("- E_FCI_451.dat: FCI energies (451 points)")

In [ ]:
# Load full dataset
x_raw = np.loadtxt('R_451.dat')
y_raw = np.loadtxt('E_FCI_451.dat')

# Sample 16 training points
idx = np.arange(0, 451, int(451/16))
x, y = x_raw[idx], y_raw[idx]

# Create prediction grid
X = np.linspace(0.5, 5, 200)

print(f"Training data: {len(x)} points")
print(f"Prediction grid: {len(X)} points")

### Exercise 1.1: Visualize the Training Data (2 points)

In [ ]:
# TODO: Create a scatter plot of the training data
# - Plot x vs y with red 'o' markers
# - Add labels: "Atomic Distance (Angstrom)" and "Energy (Hartree)"
# - Add a title: "Molecular Energy Training Data"

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Test: Check that plot was created
assert len(plt.get_fignums()) > 0, "No figure was created"
print("✓ Visualization created successfully!")

---
# Part 2: BUILD - Implementing Kernel Functions

Kernels measure similarity between points. Let's implement the most common kernels!

## Mathematical Background

### Distance Matrix
For any two matrices $X_1$ (size $n \times d$) and $X_2$ (size $m \times d$):

$$\text{dist}^2_{ij} = ||x_{1i} - x_{2j}||^2 = \sum_{k=1}^{d}(x_{1ik} - x_{2jk})^2$$

Efficiently computed as:
$$D^2 = \sum x_1^2 \mathbf{1}^T + \mathbf{1}(\sum x_2^2)^T - 2X_1X_2^T$$

### Exercise 2.1: Implement RBF (Radial Basis Function) Kernel (5 points)

The RBF kernel (also called Squared Exponential) is:

$$K(x_1, x_2) = \sigma^2 \exp\left(-\frac{||x_1-x_2||^2}{2l^2}\right)$$

Where:
- $\sigma$ controls the vertical scale (amplitude)
- $l$ controls the horizontal scale (length scale)

In [ ]:
def RBF_kernel(x1, x2, l=1.0, sigma=1.0):
    """
    Compute RBF kernel matrix.
    
    Parameters:
    -----------
    x1 : array (n, d) - First set of points
    x2 : array (m, d) - Second set of points
    l : float - Length scale parameter
    sigma : float - Amplitude parameter
    
    Returns:
    --------
    K : array (n, m) - Kernel matrix
    """
    # TODO: Implement the RBF kernel
    # Hint: Use the efficient distance calculation shown above
    
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# Test RBF kernel
test_x1 = np.array([[0], [1], [2]])
test_x2 = np.array([[0], [1]])
K = RBF_kernel(test_x1, test_x2, l=1.0, sigma=1.0)

assert K.shape == (3, 2), f"Wrong shape: {K.shape}, expected (3, 2)"
assert np.isclose(K[0, 0], 1.0), "Diagonal should be 1.0 when sigma=1"
assert K[0, 0] > K[0, 1], "Kernel should decrease with distance"
assert np.all(K >= 0), "Kernel values should be non-negative"
print("✓ RBF kernel implemented correctly!")

### Exercise 2.2: Implement Matérn 3/2 Kernel (5 points)

$$K(x_1, x_2) = \sigma^2 \left(1 + \frac{\sqrt{3}||x_1-x_2||}{l}\right) \exp\left(-\frac{\sqrt{3}||x_1-x_2||}{l}\right)$$

This kernel produces rougher functions than RBF.

In [ ]:
def Matern_kernel(x1, x2, l=1.0, sigma=1.0):
    """
    Compute Matérn 3/2 kernel matrix.
    
    Parameters: Same as RBF_kernel
    Returns: K : array (n, m) - Kernel matrix
    """
    # TODO: Implement Matérn 3/2 kernel
    # Step 1: Compute distance matrix (use sqrt this time!)
    # Step 2: Apply the Matérn 3/2 formula
    
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# Test Matérn kernel
K_matern = Matern_kernel(test_x1, test_x2, l=1.0, sigma=1.0)

assert K_matern.shape == (3, 2), f"Wrong shape: {K_matern.shape}"
assert np.isclose(K_matern[0, 0], 1.0, atol=1e-6), "Diagonal should be ~1.0 when sigma=1"
assert K_matern[0, 0] > K_matern[0, 1], "Kernel should decrease with distance"
print("✓ Matérn 3/2 kernel implemented correctly!")

### Exercise 2.3: Visualize Kernel Behavior (3 points)

Let's see how these kernels behave differently!

In [ ]:
# TODO: Compare kernel behavior
# 1. Create distances from 0 to 3 with 100 points
# 2. Compute RBF and Matérn kernels for distance from origin
# 3. Plot both on the same figure with legend

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Test: Check that comparison plot was created
assert len(plt.get_fignums()) > 0, "No figure created"
print("✓ Kernel comparison visualized!")

---
# Part 3:  ENHANCE - Building the Full GPR Class

## Theory: Gaussian Process Regression

### Joint Distribution
$$\begin{bmatrix} f(x) \\ \boldsymbol{y}^* \end{bmatrix} \sim \mathcal{N} \left( \begin{bmatrix} \mu_f \\ \mu_y \end{bmatrix}, \begin{bmatrix} K_{ff} & K_{fy} \\ K_{fy}^T & K_{yy} \end{bmatrix} \right)$$

### Posterior Prediction
$$f(x^*) \sim \mathcal{N}(\mu^*, \Sigma^*)$$

Where:
- **Mean**: $\mu^* = K_{fy}^T (K_{ff} + \sigma^2 I)^{-1} \boldsymbol{y}$
- **Covariance**: $\Sigma^* = K_{yy} - K_{fy}^T (K_{ff} + \sigma^2 I)^{-1} K_{fy}$

### Hyperparameter Optimization (Negative Log Likelihood)
$$\mathcal{L} = \frac{1}{2}\boldsymbol{y}^T K^{-1}\boldsymbol{y} + \frac{1}{2}\log|K| + \frac{n}{2}\log(2\pi)$$

In [ ]:
# Full GPR implementation (provided)
class GPR():
    def __init__(self, kernel='RBF', optimizer='L-BFGS-B', opt_params=None, bounds=None, **kw):
        self.optimizer = optimizer
        self.kernel = getattr(self, kernel)
        self.params = {
            "l": 0.5,
            "sigma": 0.2,
            "alpha": 1e-8
        }
        self.opt_params = opt_params
        self.bounds = bounds
        for k, v in kw.items():
            self.params[k] = v
        self.is_fit = False
        self.train_X, self.train_y = None, None

    def predict(self, X, return_std=False):
        X = np.asarray(X)
        if not self.is_fit:
            mean = np.zeros(X.shape[0])
            cov = self.kernel(X, X)
            if return_std:
                return mean, np.sqrt(np.diag(cov))
            return mean, cov
        
        Kff = self.kernel(self.train_X, self.train_X)
        Kyy = self.kernel(X, X)
        Kfy = self.kernel(self.train_X, X)
        Kff_inv = np.linalg.inv(Kff + self.params["alpha"] * np.eye(len(self.train_X)))
        
        mu = Kfy.T.dot(Kff_inv).dot(self.train_y).ravel()
        cov = Kyy - Kfy.T.dot(Kff_inv).dot(Kfy)
        if return_std:
            return mu, np.sqrt(np.diag(cov))
        return mu, cov

    def RBF(self, x1, x2):
        dist_matrix = np.sum(x1**2, 1).reshape(-1, 1) + np.sum(x2**2, 1) - 2 * np.dot(x1, x2.T)
        return self.params["sigma"]**2 * np.exp(-0.5 / self.params["l"]**2 * dist_matrix)
    
    def Exponential(self, x1, x2):
        dist_matrix = np.sum(x1**2, 1).reshape(-1, 1) + np.sum(x2**2, 1) - 2 * np.dot(x1, x2.T)
        dist_matrix = dist_matrix**0.5
        return self.params['sigma']**2 * np.exp(-1/self.params['l'] * dist_matrix)
    
    def Matern(self, x1, x2):
        dist_matrix = np.sum(x1**2, 1).reshape(-1, 1) + np.sum(x2**2, 1) - 2 * np.dot(x1, x2.T)
        dist_matrix = dist_matrix**0.5
        tmp1 = 1 + 3**0.5 * dist_matrix / self.params['l']
        tmp2 = np.exp(-3**0.5 * dist_matrix / self.params['l'])
        return self.params['sigma']**2 * tmp1 * tmp2
    
    def RationalQuadratic(self, x1, x2):
        dist_matrix = np.sum(x1**2, 1).reshape(-1, 1) + np.sum(x2**2, 1) - 2 * np.dot(x1, x2.T)
        alpha = self.params['kernel_alpha']
        l = self.params['l']
        inner = 1 + dist_matrix / (2 * alpha * l**2)
        return self.params['sigma']**2 * inner**(-alpha)
    
    def sample_y(self, X, n_samples=5, random_state=0):
        mean, cov = self.predict(X[:, np.newaxis])
        rs = np.random.RandomState(random_state)
        return rs.multivariate_normal(mean, cov, n_samples).T
    
    def negative_log_likelihood_loss(self, params):
        for k, v in zip(self.opt_params, params):
            self.params[k] = v
        Kyy = self.kernel(self.train_X, self.train_X) + self.params["alpha"] * np.eye(len(self.train_X))
        loss = 0.5 * self.train_y.T.dot(np.linalg.inv(Kyy)).dot(self.train_y) + \
               0.5 * np.linalg.slogdet(Kyy)[1] + \
               0.5 * len(self.train_X) * np.log(2 * np.pi)
        return loss.ravel()
    
    def fit(self, X, y):
        self.train_X = np.asarray(X)
        self.train_y = np.asarray(y)
        
        if self.optimizer and self.opt_params:
            res = minimize(self.negative_log_likelihood_loss,
                         [self.params[k] for k in self.opt_params],
                         bounds=self.bounds,
                         method=self.optimizer)
            for i, k in enumerate(self.opt_params):
                self.params[k] = res.x[i]
        self.is_fit = True

print("✓ GPR class loaded successfully!")

### Exercise 3.1: Create Visualization Function (5 points)

Complete the function to visualize prior and posterior distributions.

In [ ]:
def draw(gpr, title):
    """
    Visualize GPR prior and posterior.
    
    TODO: Complete the posterior subplot (subplot 122)
    - Fit the model with training data
    - Make predictions
    - Plot training points, mean prediction, and confidence interval
    """
    plt.figure(figsize=(20, 6), dpi=100)
    
    # PRIOR (provided)
    plt.subplot(121)
    ypred, std = gpr.predict(X[:, np.newaxis], return_std=True)
    plt.plot(X, ypred, 'k', lw=2, label='Mean')
    plt.fill_between(X, ypred+1.96*std, ypred-1.96*std, alpha=0.2, label='95% CI')
    y_samples = gpr.sample_y(X, n_samples=10)
    plt.plot(X, y_samples, alpha=0.3)
    plt.title('Prior Distribution')
    plt.xlabel('R (Angstrom)')
    plt.ylabel('E (Hartree)')
    plt.legend()
    
    # POSTERIOR (student implements)
    plt.subplot(122)
    # YOUR CODE HERE
    raise NotImplementedError()
    
    plt.suptitle(title, fontsize=16)
    plt.tight_layout()
    plt.savefig(f'{title.replace(" ", "_")}.png', dpi=150, bbox_inches='tight')
    plt.show()

In [ ]:
# Test the draw function
test_gpr = GPR(l=1.0, sigma=0.1)
draw(test_gpr, "Test GPR Visualization")
assert test_gpr.is_fit == True, "Model should be fitted"
print("✓ Visualization function works correctly!")

### Exercise 3.2: Calculate Initial Hyperparameters (2 points)

In [ ]:
# TODO: Calculate good initial guesses for l and sigma
# Hint: Use standard deviation of x for l, and y.std() / sqrt(2) for sigma

# YOUR CODE HERE
raise NotImplementedError()

print(f"Initial length scale (l): {l:.4f}")
print(f"Initial sigma: {sigma:.6f}")

In [ ]:
# Test initial parameters
assert 0.5 < l < 3.0, f"Length scale seems wrong: {l}"
assert 0.01 < sigma < 0.1, f"Sigma seems wrong: {sigma}"
print("✓ Initial parameters calculated correctly!")

---
# Part 4:  COMPARE - Testing Different Kernels

Now let's compare how different kernels perform on our molecular data!

## 4.1: RBF Kernel

In [ ]:
gpr_rbf = GPR(opt_params='l sigma alpha'.split(),
              bounds=((1e-4, 1e4), (1e-4, 1e4), (1e-10, 1e4)),
              l=l, sigma=sigma, alpha=1e-6)
draw(gpr_rbf, 'RBF Kernel')

## 4.2: Matérn 3/2 Kernel

In [ ]:
gpr_matern = GPR(kernel='Matern',
                 opt_params='l sigma alpha'.split(),
                 bounds=((1e-4, 1e4), (1e-4, 1e4), (1e-10, 1e4)),
                 l=l, sigma=sigma, alpha=1e-6)
draw(gpr_matern, 'Matérn 3/2 Kernel')

## 4.3: Exponential Kernel

In [ ]:
gpr_exp = GPR(kernel='Exponential',
              opt_params='l sigma alpha'.split(),
              bounds=((1e-4, 1e4), (1e-4, 1e4), (1e-10, 1e4)),
              l=l, sigma=sigma, alpha=1e-6)
draw(gpr_exp, 'Exponential Kernel')

## 4.4: Rational Quadratic Kernel

In [ ]:
gpr_rq = GPR(kernel='RationalQuadratic',
             opt_params='l sigma alpha'.split(),
             bounds=[(1e-4, 1e4), (1e-4, 1e4), (1e-10, 1e4)],
             kernel_alpha=10,
             l=l, sigma=sigma, alpha=1e-6)
draw(gpr_rq, 'Rational Quadratic Kernel')

### Exercise 4.1: Quantitative Comparison (8 points)

Compare the kernels using Mean Squared Error (MSE) on held-out test points.

In [ ]:
# TODO: Calculate MSE for each kernel on the full dataset
# 1. Get predictions for all points in x_raw
# 2. Calculate MSE = mean((y_pred - y_true)^2)
# 3. Store results in a dictionary
# 4. Print comparison table

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Test: Check that comparison was performed
assert 'results' in locals(), "Results dictionary not created"
assert len(results) == 4, "Should have 4 kernel results"
assert all('mse' in v for v in results.values()), "Missing MSE values"
print("✓ Kernel comparison completed successfully!")

---
# Part 5:  REFLECT - Analysis and Discussion

Answer the following questions based on your experiments.

### Exercise 5.1: Conceptual Understanding (10 points)

Answer each question in 2-3 sentences in the markdown cell below.

YOUR ANSWER HERE

---
#  Congratulations!

You've successfully:
- ✅ Implemented kernel functions from scratch
- ✅ Built and trained Gaussian Process Regression models
- ✅ Compared different kernels quantitatively
- ✅ Understood Bayesian inference for regression

## Next Steps
- Try implementing other kernels (Periodic, Linear)
- Experiment with different datasets
- Explore multi-dimensional inputs
- Study sparse GPR for large datasets

## References
- Rasmussen & Williams: "Gaussian Processes for Machine Learning" (2006)
- Murphy: "Probabilistic Machine Learning: Advanced Topics" (2023)

---